In [2]:
## Importar biblioteca do Earth Engine
import ee 
ee.Initialize()

## Importar biblioteca de correção geométrica do AROSICS
from arosics import COREG 

## getInfo = request metadado para o servidor GEE

In [3]:
## Definir parâmetros da coleção 
startDate = ee.Date('2019-09-05')                               ## Data inicial
endDate = ee.Date('2019-10-15')                                 ## Data final 
geom = ee.Geometry.Point(-61.5775286643,-8.4852883834)          ## Ponto de referência espacial 
region = geom.buffer(20000).bounds().getInfo()['coordinates']   ## Extent da imagem (20km ao redor do ponto de referência)

In [4]:
## Importar uma coleção Sentinel
sentinel_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(geom).filterDate(startDate,endDate).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',100))
print ('collection structure :', type(sentinel_collection))
print ('list length:', sentinel_collection.size().getInfo())

## Passar os metadados de acesso para uma lista JSON
sentinel_list = sentinel_collection.toList(sentinel_collection.size().getInfo())
print ('eg metadata :', sentinel_list.get(1).getInfo())


collection structure : <class 'ee.imagecollection.ImageCollection'>
list length: 7
eg metadata : {'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32720', 'crs_transform': [60, 0, 600000, 0, -60, 9100000]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32720', 'crs_transform': [10, 0, 600000, 0, -10, 9100000]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32720', 'crs_transform': [10, 0, 600000, 0, -10, 9100000]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32720', 'crs_transform': [10, 0, 600000, 0, -10, 9100000]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 

In [76]:
## Carregar imagens
ref_image = ee.Image(sentinel_list.get(2))  ## Imagem de referência
tgt_image = ee.Image(sentinel_list.get(1))  ## Imagem target

In [79]:
CR = COREG(ref_image, tgt_image)
CR.calculate_spatial_shifts()
geoRef = CR.correct_shifts()

ValueError: GeoArray_CoReg parameter 'arg' takes only string, np.ndarray or GeoArray(and subclass) instances. Got <class 'ee.image.Image'>.